In [43]:
import polars as pl
import altair as alt
import random

In [44]:
eobs = {
    "date": ["2021-01-04","2021-01-05","2021-01-06","2021-01-07","2021-01-08","2021-01-11","2021-01-12","2021-01-13","2021-01-14","2021-01-15","2021-01-19","2021-01-20"],
    "vti": [190.048508,191.583771,193.307281,196.407532,197.358429,196.179733,196.873093,197.091003,196.902817,195.189224,196.972137,199.458298],
    "vb": [190.040939,192.759628,198.970963,202.255219,202.175842,202.175842,205.192230,203.961853,206.521790,203.763412,206.114990,207.365189]
}
esyn = {
    "date": ["2021-01-04","2021-01-05","2021-01-06","2021-01-07","2021-01-08","2021-01-11","2021-01-12","2021-01-13","2021-01-14","2021-01-15","2021-01-19","2021-01-20"],
    "vti": [190.048508,191.583771,193.307281,196.407532,197.358429,196.179733,196.873093,197.091003,196.902817,195.189224,196.972137,199.458298],
    "vb": [190.040939,192.759628,198.970963,202.255219,202.175842,202.175842,205.192230,203.961853,206.521790,203.763412,206.114990,207.365189]
}
odf = pl.DataFrame(eobs)
sdf = pl.DataFrame(esyn)
sdf = sdf.with_columns(pl.col('vti')+random.uniform(-2,2))
sdf = sdf.with_columns(pl.col('vb')+random.uniform(-2,2))

In [45]:
def select_axis_endpoints(observed: pl.DataFrame,synthetic: pl.DataFrame,column: str):
    assert column in observed, "Column not in observed data"
    assert column in synthetic, "Column not in synthetic data"
    omin = observed.select(pl.min(column)).item()
    omax = observed.select(pl.max(column)).item()
    smin = synthetic.select(pl.min(column)).item()
    smax = synthetic.select(pl.max(column)).item()
    xmin = omin if omin < smin else smin
    xmax = omax if omax > smax else smax
    xmin -= (xmax-xmin)*0.1
    xmax += (xmax-xmin)*0.1
    return [xmin,xmax]

In [46]:
def plot_marginal_2d_numerical_numerical(observed: pl.DataFrame,synthetic: pl.DataFrame,col1: str,col2: str):
    xmin,xmax = select_axis_endpoints(observed,synthetic,col1)
    ymin,ymax = select_axis_endpoints(observed,synthetic,col2)
    
    odf = observed.to_pandas()
    oplot = alt.Chart(odf).mark_point(filled=True).encode(
        x = alt.X(col1+':Q',scale = alt.Scale(domain=[xmin,xmax])),
        y = alt.Y(col2+':Q',scale = alt.Scale(domain=[ymin,ymax])),
        color = alt.ColorValue("#000000")
    )
    
    sdf = synthetic.to_pandas()
    splot = alt.Chart(sdf).mark_point(filled=True).encode(
        x = alt.X(col1+':Q',scale = alt.Scale(domain=[xmin,xmax])),
        y = alt.Y(col2+':Q',scale = alt.Scale(domain=[ymin,ymax])),
        color = alt.ColorValue("#f28e2b")
    )
    return oplot+splot

plot_marginal_2d_numerical_numerical(odf,sdf,'vti','vb')


alt.LayerChart(...)